In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
!echo "Copying Data Locally (Age Regression)"
!tar xf "/content/drive/My Drive/ML4MI_BOOTCAMP_DATA/AgeRegressionChallenge.tar" --directory /home/


In [ ]:
from matplotlib import pyplot as plt
import numpy as np
import h5py

Load training, validation, and testing data.
Convert to nummpy array, add singleton dimension in channel position (1 channel -- grayscale). Edit path as needed. PyTorch expects images in the  [batch, channel, dim1, dim2] format (channels first).

In [ ]:
datapath = '/home/AgeRegressionChallenge/Data/Pneumothorax.h5'

with h5py.File(datapath,'r') as f:
    X_test = np.array(f.get('input_test')).astype(np.float32)[:,np.newaxis,:,:]
    Y_test = np.array(f.get('target_test')).astype(np.float32)[:,np.newaxis]
    X_train = np.array(f.get('input_train')).astype(np.float32)[:,np.newaxis,:,:] 
    Y_train = np.array(f.get('target_train')).astype(np.float32)[:,np.newaxis] 
    X_val =  np.array(f.get('input_val')).astype(np.float32)[:,np.newaxis,:,:]  
    Y_val = np.array(f.get('target_val')).astype(np.float32)[:,np.newaxis]  

For PyTorch we will need a data loader. This is just a simple loader without augmentation. 

In [ ]:
import torch
class Dataset(torch.utils.data.Dataset):

  def __init__(self, x, y):
        self.x = x
        self.y = y

  def __len__(self):
        return self.y.shape[0]

  def __getitem__(self, idx):
        return self.x[idx], self.y[idx]

# Create datasets
dataset_train = Dataset( X_train, Y_train)
dataset_val = Dataset( X_val, Y_val)

# Create data loader to handle shuffling, batching, etc
train_generator = torch.utils.data.DataLoader(dataset_train, batch_size=32, shuffle=True)
val_generator = torch.utils.data.DataLoader(dataset_val, batch_size=32, shuffle=False)


## <font color='red'>PyTorch Model</font> 
This is a based on a series of convolutions followed by fully connected layers (similar to ResNet) 

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class ResBlock(torch.nn.Module):
  '''Residual Block with a shortcut
  '''
  def __init__(self, in_channels, out_channels):
    super(ResBlock, self).__init__()

    # First convolution
    self.conv1 = nn.Conv2d(in_channels=in_channels, out_channels=out_channels, kernel_size=(3,3), padding='same')
    self.norm1 = nn.BatchNorm2d(out_channels)
    self.act1 = torch.nn.ReLU()

    # Second convolution
    self.conv2 = nn.Conv2d(in_channels=out_channels, out_channels=out_channels, kernel_size=(3,3), padding='same')
    self.norm2 = nn.BatchNorm2d(out_channels)
    
    # Shortcut 
    self.convs =  nn.Conv2d(in_channels=in_channels, out_channels=out_channels, kernel_size=(1,1), padding='same')
    self.act2 = torch.nn.ReLU()

  def forward(self, x):
    shortcut = self.convs(x)
    x = self.conv1(x)
    x = self.norm1(x)
    x = self.act1(x)
    x = self.conv2(x)
    x = self.norm2(x)
    x = x + shortcut
    x = self.act2(x)
    return x

class AgeNet(torch.nn.Module):

  def __init__(self, depth=6, initial_features=32, input_size=256):
    super(AgeNet, self).__init__()

    in_channels = 1
    out_channels = initial_features
    im_size = 256

    # This will build the list of operators, similar to keras sequential which each being a convolution followed by activation
    layers = []
    for l in range(depth):
      layers.append(ResBlock(in_channels, out_channels))
      in_channels = out_channels
      layers.append(nn.MaxPool2d(kernel_size=2))
      im_size = im_size // 2
          
    # This stores the layers in a list that will be tracked for backpropogation
    self.convolutions = nn.ModuleList(layers)

    # Fully connected layers
    self.fc1 = nn.Linear( im_size**2 * out_channels, 512)
    self.act1 = nn.ReLU()
    self.fc2 = nn.Linear( 512, 1)

  def forward(self, image):
    for l in self.convolutions:
      image = l(image)
    
    # Flatten
    image = image.view(image.size(0), -1)
    
    # Fully connected
    image = self.fc1(image)
    image = self.act1(image)
    image = self.fc2(image)

    return image


# To run the model we need to create an object based on those class definitions above
age_model = AgeNet()

# Models have a device they run on. We need to put the model on the gpu
age_model = age_model.cuda()

# Torch summary enables similar formatting to Keras
from torchsummary import summary
summary(age_model, (1,256,256))

## <font color='red'>Run the model fitting</font> 


Fit the model. Modify the epochs/batch_size as needed. 

In [ ]:
# Define an optimizer 
optimizer = torch.optim.Adam( age_model.parameters(), lr=1e-4)

# Define a loss function
loss_fcn = nn.MSELoss()

# Get a device
use_cuda = torch.cuda.is_available()
device = torch.device("cuda:0" if use_cuda else "cpu")

# Ensure the model weights are on the desired device
age_model.to(device)

# Define number of epochs
n_epochs = 50

# Empty list to store losses over epochs
train_losses = []
val_losses = []

# Epoch loop
for epoch in range(n_epochs):

  # Put the model in train mode
  age_model.train()

  # Loop over training batches
  train_loss_avg = 0.0
  for x, y in train_generator:
    
    # Move data to the GPU
    x = x.to(device)
    y = y.to(device)

    # Zero out the gradients
    optimizer.zero_grad()

    # Forward pass
    y_guess = age_model(x)

    # Loss
    loss = loss_fcn( y_guess, y)

    # Store loss 
    train_loss_avg += loss.detach()

    # Backwards with perform back propogation to get weights
    loss.backward()
    
    # Take a gradient descent step
    optimizer.step()

  # Store the average loss
  train_loss_avg /= len(train_generator)
  train_losses.append(train_loss_avg)

  # Switch to eval mode (weights fixed)
  age_model.eval()

  # Loop eval batches
  val_loss_avg = 0.0
  for count, (x, y) in enumerate(val_generator):

    # Move data to the GPU
    x = x.to(device)
    y = y.to(device)
   
    # Forward pass
    y_guess = age_model(x)

    # Loss
    loss = loss_fcn( y_guess, y)
    val_loss_avg += loss.detach()

  # Store the average loss
  val_loss_avg /= len(val_generator)
  val_losses.append(val_loss_avg)

  print(f'Epoch = {epoch} Loss = {train_loss_avg}, Val Loss = {val_loss_avg}')

Plot the training/validation loss

In [ ]:
plt.figure()
plt.semilogy(train_losses,'bo', label='Training mse')
plt.semilogy(val_losses,'b', label='Validation mse')
plt.legend()
plt.show()

This is the code to use to evaluate your network -- don't change it. Take a screen shot of the output to submit to the competition! (everyone should submit)

In [ ]:
Y_pred = []
for idx in range(X_test.shape[0]):
  x = X_test[idx][np.newaxis,...]
  x = torch.tensor(x)
  
  # Move data to the GPU
  x = x.to(device)
  
  # Forward pass
  Y_pred.append(age_model(x).detach().cpu().numpy())
Y_pred = np.array(Y_pred)

Y_pred = np.squeeze(Y_pred)  #remove the singleton dimension for analysis
Y_test = np.squeeze(Y_test)  
plt.scatter(Y_test, Y_pred, s=2)
plt.xlabel('True age')
plt.ylabel('Predicted age')
plt.show()
corr = np.corrcoef(Y_pred, Y_test)   #get correlation matrix
print("Correlation coefficient: " + str(corr[0,1]))
